In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse.linalg 
import scipy
import pandas as pd
import copy
import seaborn as sns
import pickle
from scipy import sparse

# 矩阵法计算ECI

# example

--|steel|oil|chip|bean|clothes
--|--|--|--|--|--
USA|1|1|1|1|0
China|1|0|0|1|1
Viet Nam|0|0|0|1|1

In [16]:
# 实际例子
product_name = ["steel","oil","chip","bean","clothes"]
country_name = ["USA","China","Viet Nam"]
X = ['USA', 'USA', 'USA', 'USA', 'China', 'China', 'China', 'Viet Nam', 'Viet Nam']
Y = ['steel', 'oil', 'chip', 'bean', 'steel', 'bean', 'clothes', 'bean', 'clothes']

In [17]:
L_x = list(set(X))
L_y = list(set(Y))
x2id = dict(zip(L_x,range(len(L_x))))
y2id = dict(zip(L_y,range(len(L_y))))
x = [x2id[xi] for xi in X]
y = [y2id[yi] for yi in Y]

In [21]:
data = [1]*len(x)
m,n = len(x2id),len(y2id)
M = sparse.csc_matrix((data,(x,y)),shape=(m,n))
M = np.matrix(M.toarray())
eci,pci = Get_eci_pci(M)
eci = Get_z_score(eci)
pci = Get_z_score(pci)
print(L_x)
print(eci)
print('='*50)
print(L_y)
print(pci)

['USA', 'Viet Nam', 'China']
[ 1.37652932 -0.96911209 -0.40741722]
['steel', 'bean', 'oil', 'chip', 'clothes']
[-0.0316507  -0.63750372  1.08360666  1.08360666 -1.4980589 ]


# 稀疏矩阵方法

In [11]:
data = [1]*len(x)
m,n = len(x2id),len(y2id)
M = sparse.csc_matrix((data,(x,y)),shape=(m,n))
eci,pci = Get_eci_pci_sparse(M)
eci = Get_z_score(eci)
pci = Get_z_score(pci)
print(L_x)
print(eci)
print('='*50)
print(L_y)
print(pci)

['USA', 'Viet Nam', 'China']
[ 1.37652932 -0.96911209 -0.40741722]
['steel', 'bean', 'oil', 'chip', 'clothes']
[-0.0316507  -0.63750372  1.08360666  1.08360666 -1.4980589 ]


# 利用 ecomplexity 计算


In [9]:
#构建原始数据
from ecomplexity import ecomplexity
from ecomplexity import proximity
# Calculate complexity
trade_cols = {'time':'year', 'loc':'name', 'prod':'skill', 'val':'value'}
Name=[]
Skill=[]
time=[]
value=[]

time=[2019]*len(x)    
value=[1.]*len(x)   
SData=pd.DataFrame({"year":time,'name':x,"skill":y,"value":value})
cdata = ecomplexity(SData, trade_cols,presence_test="manual")#不使用过滤函数

2019


In [10]:
cdata

,name,skill,value,year,diversity,ubiquity,mcp,eci,pci,density,coi,cog
0,0,0,1.0,2019,4.0,2.0,1.0,1.376529,0.715973,0.842105,-1.366512,-0.000000
1,0,1,1.0,2019,4.0,3.0,1.0,1.376529,-0.269341,0.777778,-1.366512,-0.000000
2,0,2,1.0,2019,4.0,1.0,1.0,1.376529,2.529743,1.000000,-1.366512,0.000000
3,0,3,1.0,2019,4.0,1.0,1.0,1.376529,2.529743,1.000000,-1.366512,0.000000
4,0,4,0.0,2019,4.0,2.0,0.0,1.376529,-1.668883,0.538462,-1.366512,-0.890543
5,1,0,0.0,2019,2.0,2.0,0.0,-0.969112,0.715973,0.368421,0.367845,0.944658
6,1,1,1.0,2019,2.0,3.0,1.0,-0.969112,-0.269341,0.555556,0.367845,0.000000
7,1,2,0.0,2019,2.0,1.0,0.0,-0.969112,2.529743,0.117647,0.367845,1.673626
8,1,3,0.0,2019,2.0,1.0,0.0,-0.969112,2.529743,0.117647,0.367845,1.673626
9,1,4,1.0,2019,2.0,2.0,1.0,-0.969112,-1.668883,0.769231,0.367845,0.000000


# check 完成
利用稀疏矩阵计算出来的eci与ecomplexity计算得到的eci完全一致